# Entrega 3: Preparação dos dados

Nesse notebook/relatório iremos realizar e justificar o processo em cada etapa da preparação dos dados da base a qual nosso grupo ficou responsável. Para assim, na próxima entrega, testar modelos de machine learning de classificação.

### Grupo
 - Nilo Bemfica (nbmcd)
 - Pedro Didier (pdm)
 - Pedro Tenório (ptl)

# Imports e Loads

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
%matplotlib inline

c:\Users\Pedro\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
df = pd.read_csv("data.csv")

# Visualizando as features

Dicionário de colunas:

gender: 0 (unknown), 1 (male), 2 (female)  
C_api: gender extracted from WikiMedia API, codes as female / male / unknown  
C_man: gender extracted from content coding, coded as 1 (male) / 2 (female) / 3 (unknown)  
E_NEds: I index of stratum IJ (0,1,2,3)  
E_Bpag: J index of stratum IJ (0,1,2,3)  
firstDay: first edition in the Spanish Wikipedia (YYYYMMDDHHMMSS)  
lastDay: last edition in the Spanish Wikipedia (YYYYMMDDHHMMSS)  
NEds: total number of editions  
NDays: number of days (lastDay-firstDay+1)  
NActDays: number of days with editions  
NPages: number of different pages edited  
NPcreated: number of pages created  
pagesWomen: number of edits in pages related to women  
wikiprojWomen: number of edits in WikiProjects related to women  
ns_user: number of edits in namespace user  
ns_wikipedia: number of edits in namespace wikipedia  
ns_talk: number of edits in namespace talk  
ns_userTalk: number of edits in namespace user talk  
ns_content: number of edits in content pages  
weightIJ: correcting weight for stratum IJ  
NIJ: number of elements in stratum IJ

* É importante manter em mente esse dicionário pois ele vai guiar o processamento da base.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         4746 non-null   int64  
 1   C_api          4746 non-null   object 
 2   C_man          4746 non-null   int64  
 3   E_NEds         4746 non-null   int64  
 4   E_Bpag         4746 non-null   int64  
 5   firstDay       4746 non-null   int64  
 6   lastDay        4746 non-null   int64  
 7   NEds           4746 non-null   int64  
 8   NDays          4746 non-null   int64  
 9   NActDays       4746 non-null   int64  
 10  NPages         4746 non-null   int64  
 11  NPcreated      4746 non-null   int64  
 12  pagesWomen     4746 non-null   int64  
 13  wikiprojWomen  4746 non-null   int64  
 14  ns_user        4746 non-null   int64  
 15  ns_wikipedia   4746 non-null   int64  
 16  ns_talk        4746 non-null   int64  
 17  ns_userTalk    4746 non-null   int64  
 18  ns_conte

In [4]:
df.head()

,gender,C_api,C_man,E_NEds,E_Bpag,firstDay,lastDay,NEds,NDays,NActDays,...,NPcreated,pagesWomen,wikiprojWomen,ns_user,ns_wikipedia,ns_talk,ns_userTalk,ns_content,weightIJ,NIJ
0,1,male,1,2,2,20170527205915,20170721044501,543,56,43,...,4,0,0,91,28,6,76,324,0.915024,978
1,0,unknown,3,3,1,20110301072441,20170731213735,2764,2345,514,...,7,0,0,100,249,183,646,1526,0.661673,477
2,1,male,1,0,2,20060907204302,20140911191722,57,2927,25,...,0,0,0,3,0,1,3,49,0.800528,664
3,1,male,1,1,2,20121003144916,20121208180528,104,67,5,...,2,0,0,20,1,2,2,78,1.027717,841
4,0,unknown,3,1,1,20070311125035,20141106121057,184,2798,27,...,0,0,0,26,10,5,24,112,0.997535,994


# Preparação dos Dados

## Checando valores nulos e linhas duplicadas

Essa etapa foi realizada durante o EDA, mas é essencial na parte de preparação dos dados. Dito isso, nosso dataset tem 0 valores nulos e 0 linhas duplicadas.

In [5]:
# generate preview of entries with null values
if df.isnull().any(axis=None):
    print("\nPreview of data with null values:\nxxxxxxxxxxxxx")
    print(df[df.isnull().any(axis=1)].head(3))
    missingno.matrix(df)
    plt.show()
else:
    print("No null entries found")

No null entries found


In [6]:
# generate count statistics of duplicate entries
if len(df[df.duplicated()]) > 0:
    print("No. of duplicated entries: ", len(df[df.duplicated()]))
    print(df[df.duplicated(keep=False)].sort_values(by=list(df.columns)).head())
else:
    print("No duplicated entries found")

No duplicated entries found


## Visão geral das features

Com base no que foi pontuado no relatório de EDA, temos: 

 - gender: Codificada conforme mencionado (0 para desconhecido, 1 para masculino e 2 para feminino).
 - C_api: Esta coluna possui valores de gênero extraídos da API do WikiMedia.
 - C_man: Esta coluna possui valores de gênero extraídos da codificação de conteúdo.
 - E_NEds e E_Bpag: São índices de estrato.
 - firstDay e lastDay: Representam datas em formato YYYYMMDDHHMMSS.
 - Várias colunas como NEds, NDays, NActDays, etc.: Representam diferentes métricas relacionadas às edições feitas no Wikipedia.
 - weightIJ: É um peso corretivo para o estrato IJ.
 - NIJ: É o número de elementos no estrato IJ.

## Definindo um tratamento proposto para cada coluna:

 - *gender*: Manter do jeito que está
 - *C_api*: Converter em códigos numéricos consistentes com a coluna gender (0 para desconhecido, 1 para masculino, 2 para feminino).
 - *C_man*: Para manter a consistência com a coluna gender, vamos reajustar os códigos para (0, 1, 2) em vez de (1, 2, 3).

Vale a pena constatar que provavelmente para o modelo só usaremos a coluna gender como label e descartaremos *C_api* e *C_man*.

 - *E_NEds* e *E_Bpag*: Estas são colunas categóricas de estratificação com base em outras features.
 - *firstDay* e *lastDay*: Assim como no EDA, estas colunas serão convertidas para o formato de data padrão do Python (datetime) para facilitar cálculos posteriores.
 - *NEds*, *NDays*, *NActDays*, *NPages*, *NPcreated*, *pagesWomen*, *wikiprojWomen*, *ns_user*, *ns_wikipedia*, *ns_talk*, *ns_userTalk*, *ns_content*: Estas são colunas numéricas relacionadas às edições. Na etapa de EDA percebemos muitos outliers. Vamos aderessar isso agora.
 - *weightIJ*: Esta é uma coluna numérica, mas que representa um peso.
 - *NIJ*: Também é uma coluna numérica, mas que representa um índice.

## A situação dos outliers nas colunas numéricas e como vamos tratá-los

Vamos olhar novamente o describe do dataset, focando nas colunas numéricas.

In [7]:
edit_columns = ["NEds", "NDays", "NActDays", "NPages", "NPcreated", "pagesWomen", "wikiprojWomen",
                "ns_user", "ns_wikipedia", "ns_talk", "ns_userTalk", "ns_content"]

statistics = df[edit_columns].describe()
statistics

,NEds,NDays,NActDays,NPages,NPcreated,pagesWomen,wikiprojWomen,ns_user,ns_wikipedia,ns_talk,ns_userTalk,ns_content
count,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000
mean,2029.969448,2036.607880,183.162663,689.451960,43.479140,0.438896,0.439949,74.372946,74.368310,49.947745,96.081753,1521.886641
std,7793.300833,1336.119914,374.034481,3355.302483,297.395507,5.327440,17.832244,246.407233,560.782479,215.554281,545.025818,6099.009235
min,50.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,95.000000,835.250000,24.000000,29.000000,1.000000,0.000000,0.000000,4.000000,0.000000,0.000000,1.000000,61.000000
50%,218.000000,2035.500000,53.000000,68.000000,4.000000,0.000000,0.000000,14.000000,1.000000,4.000000,5.000000,151.000000
75%,757.750000,3146.500000,154.000000,219.750000,14.000000,0.000000,0.000000,46.000000,8.000000,19.000000,22.000000,563.750000
max,153193.000000,5349.000000,3843.000000,94142.000000,13394.000000,185.000000,949.000000,6041.000000,24392.000000,4788.000000,12350.000000,115547.000000


 - *NEds* (total number of editions): Varia entre 50 e 153193 com uma média de aproximadamente 2029.97. A grande diferença entre a média e a mediana (218) é enorme e indica a presença de outliers.
 - *NDays* (number of days): Varia entre 1 e 5349 com uma média de aproximadamente 2036.61. A distribuição parece relativamente uniforme.
 - *NActDays* (number of active days): Varia entre 1 e 3843 com uma média de aproximadamente 183.16. A presença de outliers também é indicada pela grande diferença entre a média e a mediana (53).
 - *NPages* (number of different pages edited): Varia entre 1 e 94142 com uma média de aproximadamente 689.45. A presença de outliers é presente aqui também.
 - *NPcreated* (number of pages created): A média é de 43.48, mas varia até 13394, indicando outliers.
 - *pagesWomen* e *wikiprojWomen*: A maioria dos valores é 0, indicando que muitos editores não editaram páginas relacionadas às mulheres. Existe presença de outliers que são os poucos que de fato editaram. Vamos tratar essas duas de uma maneira levemente diferente.
 - As colunas *ns_user*, *ns_wikipedia*, *ns_talk*, *ns_userTalk* e *ns_content* representam o número de edições em diferentes namespaces. Estas colunas também mostram grandes variações nos seus valores máximos, indicando a presença de outliers.

### Tratamento proposto:

Para as colunas *N_days*, *NActDays*, *NPages*, *NPcreated*, *ns_user*, *ns_wikipedia*, *ns_talk*, *ns_userTalk* e *ns_content* vamos utilizar uma técnica baseada em IQR para limitar os valores mínimos e máximos.  

As colunas *pagesWomen* e *wikiprojWomen* serão transformadas em colunas binárias uma vez que tratar com a técnica anterior poderia eliminar a informação por completo devido aos poucos valores diferentes de zero em cada uma.  

As demais colunas numéricas serão mantidas da maneira que estão pois não indicaram problemas.

## Transformações

Enfim, baseado em tudo que foi listado, vamos transformar os dados.

Convertendo a coluna 'C_api' para códigos numéricos

In [8]:
df['C_api'] = df['C_api'].map({'unknown': 0, 'male': 1, 'female': 2})

Adaptando os códigos de gênero da coluna 'C_man'

In [9]:
df['C_man'] = df['C_man'] - 1

Vamos transformar a coluna de firstDay e lastDay de fato em colunas de data

In [10]:
df["firstDay"] = pd.to_datetime(df["firstDay"], format='%Y%m%d%H%M%S')
df["lastDay"] = pd.to_datetime(df["lastDay"], format='%Y%m%d%H%M%S')

Transformando as colunas *paesWomen* e *wikiprojWomen* em colunas binárias

In [11]:
df["pagesWomen"] = df["pagesWomen"].apply(lambda x: 1 if x > 0 else 0)
df["wikiprojWomen"] = df["wikiprojWomen"].apply(lambda x: 1 if x > 0 else 0)

Tratando os outliers com uma técnica baseada em IQR.

In [12]:
# Função para tratar outliers usando a técnica IQR
def treat_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Substituindo os outliers pelos limites do IQR
    df[column] = df[column].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))
    return df

# Tratando outliers nas colunas indicadas
columns_to_treat = ["NEds", "NActDays", "NPages", "NPcreated", "ns_user", "ns_wikipedia", "ns_talk", "ns_userTalk", "ns_content"]

for col in columns_to_treat:
    df = treat_outliers_iqr(df, col)

##  Verificando estatísticas após o tratamento

In [13]:
statistics_after_treatment = df.describe()
statistics_after_treatment

,gender,C_api,C_man,E_NEds,E_Bpag,NEds,NDays,NActDays,NPages,NPcreated,pagesWomen,wikiprojWomen,ns_user,ns_wikipedia,ns_talk,ns_userTalk,ns_content,weightIJ,NIJ
count,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000
mean,0.737042,0.426254,1.082807,1.484197,1.646228,534.669617,2036.607880,107.643911,154.051043,9.575327,0.053940,0.011589,31.358618,5.473030,12.894753,15.104720,394.229825,1.000000,867.148546
std,0.585355,0.566484,0.964978,1.099795,1.079263,609.001757,1336.119914,115.380816,174.464930,11.993828,0.225923,0.107037,36.811117,7.467627,17.019181,19.135006,465.118926,0.325763,325.933076
min,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.411985,297.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,95.000000,835.250000,24.000000,29.000000,1.000000,0.000000,0.000000,4.000000,0.000000,0.000000,1.000000,61.000000,0.831954,664.000000
50%,1.000000,0.000000,2.000000,1.000000,2.000000,218.000000,2035.500000,53.000000,68.000000,4.000000,0.000000,0.000000,14.000000,1.000000,4.000000,5.000000,151.000000,0.997535,917.000000
75%,1.000000,1.000000,2.000000,2.000000,3.000000,757.750000,3146.500000,154.000000,219.750000,14.000000,0.000000,0.000000,46.000000,8.000000,19.000000,22.000000,563.750000,1.057149,994.000000
max,2.000000,2.000000,2.000000,3.000000,3.000000,1751.875000,5349.000000,349.000000,505.875000,33.500000,1.000000,1.000000,109.000000,20.000000,47.500000,53.500000,1317.875000,1.865008,1596.000000


NEds, NActDays, NPages, NPcreated, ns_user, ns_wikipedia, ns_talk, ns_userTalk e ns_content, agora têm valores máximos significativamente reduzidos em comparação com as estatísticas anteriores, indicando que os outliers foram tratados.

## Conclusão e próximos passos:

 - Os dados agora estão mais consistentes e prontos para utilização de modelos.
 - A conversão das colunas de data para o formato datetime facilitará qualquer análise temporal subsequente.
 - Talvez seja interessante aplicar normalização dos dados por z-score para alguns modelos de classificação.

In [14]:
df.to_csv("homologated_data.csv", index=False)